# Methodology Overview

All of the indices use ordinal indicators where policies a ranked on a simple numerical scale. The project also records five non-ordinal indicators – E3, E4, H4, H5 and M1 – but these are not used in our index calculations. As of August 2021, E3, E4, and H4 will no longer be updated. The data remains present in the CSV files.

Some indicators – C1-C7, E1, H1, H6 and H7 – have an additional binary flag variable that can be either 0 or 1. For C1-C7, H1 and H6 this corresponds to the geographic scope of the policy. For E1, this flag variable corresponds to the sectoral scope of income support. For H7, this flag variable corresponds to whether the individual or government is funding the vaccination.

The [codebook](codebook.md) has details about each indicator and what the different values represent.

Because different indicators (_j_) have different maximum values (_N<sub>j</sub>_) in their ordinal scales, and only some have flag variables, each sub-index score must be calculated separately. The different indicators are:

| Indicator | Max. value (_N<sub>j</sub>_) | Flag? (_F<sub>j</sub>_) |
| --- | --- | --- |
| C1 | 3 (0, 1, 2, 3) | yes=1 |
| C2 | 3 (0, 1, 2, 3) | yes=1 |
| C3 | 2 (0, 1, 2) | yes=1 |
| C4 | 4 (0, 1, 2, 3, 4) | yes=1 |
| C5 | 2 (0, 1, 2) | yes=1 |
| C6 | 3 (0, 1, 2, 3) | yes=1 |
| C7 | 2 (0, 1, 2) | yes=1 |
| C8 | 4 (0, 1, 2, 3, 4) | no=0 |
| E1 | 2 (0, 1, 2) | yes=1 |
| E2 | 2 (0, 1, 2) | no=0 |
| H1 | 2 (0, 1, 2) | yes=1 |
| H2 | 3 (0, 1, 2, 3) | no=0 |
| H3 | 2 (0, 1, 2) | no=0 |
| H6 | 4 (0, 1, 2, 3, 4) | yes=1 |
| H7 | 5 (0, 1, 2, 3, 4, 5) | yes=1 |
| H8 | 3 (0, 1, 2, 3) | yes=1 |

Each sub-index score (_I_) for any given indicator (_j_) on any given day (_t_), is calculated by the function described in equation 2 based on the following parameters:

- the maximum value of the indicator (_N<sub>j</sub>_)
- whether that indicator has a flag (_F<sub>j</sub>_=1 if the indicator has a flag variable, or 0 if the indicator does not have a flag variable)
- the recorded policy value on the ordinal scale (_v<sub>j,t</sub>_)
- the recorded binary flag for that indicator (_f<sub>j,t</sub>_)

This normalises the different ordinal scales to produce a sub-index score between 0 and 100 where each full point on the ordinal scale is equally spaced. For indicators that do have a flag variable, if this flag is recorded as 0 (ie if the policy is geographically _targeted_ or for E1 if the support only applies to _informal sector workers_) then this is treated as a half-step between ordinal values.

Note that the database only contains flag values if the indicator has a non-zero value. If a government has no policy for a given indicator (ie the indicator equals zero) then the corresponding flag is blank/null in the database. For the purposes of calculating the index, this is equivalent to a sub-index score of zero. In other words, _I<sub>j,t</sub>_=0 if _v<sub>j,t</sub>_=0.

![sub-index score equation](https://latex.codecogs.com/png.latex?%282%29%5Cqquad%20I_%7Bj%2Ct%7D%3D100%5Cfrac%7Bv_%7Bj%2Ct%7D-0.5%28F_%7Bj%7D-f_%7Bj%2Ct%7D%29%7D%7BN_%7Bj%7D%7D)

(_if v<sub>j,t</sub>=0 then the function F<sub>j</sub>-f<sub>j,t</sub> is also treated as 0, see paragraph above._)

## References

https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/index_methodology.md

https://github.com/OxCGRT/covid-policy-tracker/blob/master/documentation/codebook.md

# Code for calcuating Closure Index

## Import required packages

In [1]:
import pandas as pd
from datetime import datetime
import calendar

## Load Data

Load the data and change the columns names for easier processing. Select only the required columns from the csv and drop the rest

In [2]:
dataPath = "/home/nilanjan/Juypter_Workspaces/Shibu/OxCGRT_latest.csv"

## Have custom column names
headerNames = "countryname,countrycode,regionname,regioncode,jurisdiction,date,c1_school_closing,c1_flag,c2_workplace_closing,c2_flag,c3_cancel_public_events,c3_flag,c4_restrictions_on_gatherings,c4_flag,c5_close_public_transport,c5_flag,c6_stay_at_home_requirements,c6_flag,c7_restrictions_on_internal_movement,c7_flag,c8_international_travel_controls,e1_income_support,e1_flag,e2_debt_contract_relief,e3_fiscal_measures,e4_international_support,h1_public_information_campaigns,h1_flag,h2_testing_policy,h3_contact_tracing,h4_emergency_investment_in_healthcare,h5_investment_in_vaccines,h6_facial_coverings,h6_flag,h7_vaccination_policy,h7_flag,h8_protection_of_elderly_people,h8_flag,m1_wildcard,v1_vaccine_prioritisation_summary,v2a_vaccine_availability_summary,v2b_vaccine_age_eligibility,v2c_vaccine_age_eligibility,v2d_clinically_vulnerable_non_elderly,v2e_education,v2f_frontline_workers_non_healthcare,v2g_frontline_workers_healthcare,v3_vaccine_financial_support_summary,confirmedcases,confirmeddeaths,stringencyindex,stringencyindexfordisplay,stringencylegacyindex,stringencylegacyindexfordisplay,governmentresponseindex,governmentresponseindexfordisplay,containmenthealthindex,containmenthealthindexfordisplay,economicsupportindex,economicsupportindexfordisplay"
colNames = headerNames.split(",")

#no header
#skip a row
oxford_df = pd.read_csv(
  dataPath,
  header = None,
  skiprows = 1,
  sep = ',',
  names = colNames
)

## Select only required columns
cleaned_df = oxford_df[["countryname","countrycode","regionname","regioncode","date","c1_school_closing","c1_flag","c2_workplace_closing","c2_flag","c3_cancel_public_events","c3_flag","c4_restrictions_on_gatherings","c4_flag","c5_close_public_transport","c5_flag","c6_stay_at_home_requirements","c6_flag","c7_restrictions_on_internal_movement","c7_flag","c8_international_travel_controls"]]

/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## Break data into year, month and day

In [3]:
## break down date into components

def get_year(input_dt):
    normalized=str(input_dt)
    dt = datetime.strptime(normalized, '%Y%m%d')
    return dt.year

def get_month(input_dt):
    normalized=str(input_dt)
    dt = datetime.strptime(normalized, '%Y%m%d')
    return calendar.month_name[dt.month]

def get_day(input_dt):
    normalized=str(input_dt)
    dt = datetime.strptime(normalized, '%Y%m%d')
    return dt.day

cleaned_df['year'] = cleaned_df['date'].apply(get_year)
cleaned_df['month'] = cleaned_df['date'].apply(get_month)
cleaned_df['day'] = cleaned_df['date'].apply(get_day)

# rearrage the columns
base_df = cleaned_df[["countryname","countrycode","regionname","regioncode","date","year","month","day","c1_school_closing","c1_flag","c2_workplace_closing","c2_flag","c3_cancel_public_events","c3_flag","c4_restrictions_on_gatherings","c4_flag","c5_close_public_transport","c5_flag","c6_stay_at_home_requirements","c6_flag","c7_restrictions_on_internal_movement","c7_flag","c8_international_travel_controls"]]

/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

pandas.core.frame.DataFrame

## See input sample data 

In [4]:
base_df.head()

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c3_flag,c4_restrictions_on_gatherings,c4_flag,c5_close_public_transport,c5_flag,c6_stay_at_home_requirements,c6_flag,c7_restrictions_on_internal_movement,c7_flag,c8_international_travel_controls
0,Aruba,ABW,NaN,NaN,20200101,2020,January,1,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
1,Aruba,ABW,NaN,NaN,20200102,2020,January,2,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
2,Aruba,ABW,NaN,NaN,20200103,2020,January,3,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
3,Aruba,ABW,NaN,NaN,20200104,2020,January,4,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0
4,Aruba,ABW,NaN,NaN,20200105,2020,January,5,0.0,NaN,...,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0,NaN,0.0


## Index calculation methods

Calcuate indexes for columns from C1 through C2

In [5]:
#### Calculate Index value for C1
def calculate_c1_index(x):
    if (x['c1_school_closing']==0 or x['c1_school_closing'] != x['c1_school_closing'] or x['c1_flag'] != x['c1_flag']):
        return 0
    else:
        return round(100*((x['c1_school_closing'] - 0.5*(1 - x['c1_flag']))/3),4)

#### Calculate Index value for C2
def calculate_c2_index(x):
    if (x['c2_workplace_closing']==0 or x['c2_workplace_closing'] != x['c2_workplace_closing'] or x['c2_flag'] != x['c2_flag']):
        return 0
    else:
        return round(100*((x['c2_workplace_closing'] - 0.5*(1 - x['c2_flag']))/3),4)

#### Calculate Index value for C3
def calculate_c3_index(x):
    if (x['c3_cancel_public_events']==0 or x['c3_cancel_public_events'] != x['c3_cancel_public_events'] or x['c3_flag'] != x['c3_flag']):
        return 0
    else:
        return round(100*((x['c3_cancel_public_events'] - 0.5*(1 - x['c3_flag']))/2),4)

#### Calculate Index value for C4
def calculate_c4_index(x):
    if (x['c4_restrictions_on_gatherings']==0 or x['c4_restrictions_on_gatherings'] != x['c4_restrictions_on_gatherings'] or x['c4_flag'] != x['c4_flag']):
        return 0
    else:
        return round(100*((x['c4_restrictions_on_gatherings'] - 0.5*(1 - x['c4_flag']))/4),4)

#### Calculate Index value for C5
def calculate_c5_index(x):
    if (x['c5_close_public_transport']==0 or x['c5_close_public_transport'] != x['c5_close_public_transport'] or x['c5_flag'] != x['c5_flag']):
        return 0
    else:
        return round(100*((x['c5_close_public_transport'] - 0.5*(1 - x['c5_flag']))/2),4)

#### Calculate Index value for C6
def calculate_c6_index(x):
    if (x['c6_stay_at_home_requirements']==0 or x['c6_stay_at_home_requirements'] != x['c6_stay_at_home_requirements'] or x['c6_flag'] != x['c6_flag']):
        return 0
    else:
        return round(100*((x['c6_stay_at_home_requirements'] - 0.5*(1 - x['c6_flag']))/3),4)

#### Calculate Index value for C7
def calculate_c7_index(x):
    if (x['c7_restrictions_on_internal_movement']==0 or x['c7_restrictions_on_internal_movement'] != x['c7_restrictions_on_internal_movement'] or x['c7_flag'] != x['c7_flag']):
        return 0
    else:
        return round(100*((x['c7_restrictions_on_internal_movement'] - 0.5*(1 - x['c7_flag']))/2),4)

#### Calculate Index value for C8
def calculate_c8_index(x):
    if (x['c8_international_travel_controls']==0 or x['c8_international_travel_controls'] != x['c8_international_travel_controls']):
        return 0
    else:
        return round(100*(x['c8_international_travel_controls']/4),4)
    
## Calcualte Indices
base_df['c1_index'] = base_df.apply(calculate_c1_index,axis=1)
base_df['c2_index'] = base_df.apply(calculate_c2_index,axis=1)
base_df['c3_index'] = base_df.apply(calculate_c3_index,axis=1)
base_df['c4_index'] = base_df.apply(calculate_c4_index,axis=1)
base_df['c5_index'] = base_df.apply(calculate_c5_index,axis=1)
base_df['c6_index'] = base_df.apply(calculate_c6_index,axis=1)
base_df['c7_index'] = base_df.apply(calculate_c7_index,axis=1)
base_df['c8_index'] = base_df.apply(calculate_c8_index,axis=1)

/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

## See  sample data with indexes

In [6]:
base_df.head()

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c7_flag,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index
0,Aruba,ABW,NaN,NaN,20200101,2020,January,1,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,NaN,NaN,20200102,2020,January,2,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,NaN,NaN,20200103,2020,January,3,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,ABW,NaN,NaN,20200104,2020,January,4,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,ABW,NaN,NaN,20200105,2020,January,5,0.0,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Calculate the Closure Index

In [7]:
# closure_index
base_df['closure_index'] = base_df.apply(lambda x: round((x['c1_index'] + x['c2_index'] + x['c3_index'] + x['c4_index'] + x['c5_index'] + x['c6_index'] + x['c7_index'] + x['c8_index'])/9
,4),axis=1)

/opt/anaconda3/envs/data_science/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


## See sample data for closure index

In [8]:
# closure_index example
base_df.query('c4_restrictions_on_gatherings==2 and c4_flag==0').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
1886,Angola,AGO,NaN,NaN,20201118,2020,November,18,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407
1887,Angola,AGO,NaN,NaN,20201119,2020,November,19,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407
1888,Angola,AGO,NaN,NaN,20201120,2020,November,20,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407
1889,Angola,AGO,NaN,NaN,20201121,2020,November,21,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407
1890,Angola,AGO,NaN,NaN,20201122,2020,November,22,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407


##  Write data to excel

In [9]:
output_file_name="/home/nilanjan/Juypter_Workspaces/Shibu/country_wise_closure_index.xlsx"
sheet_name="Closure_Index"
base_df.to_excel(output_file_name,sheet_name=sheet_name)

# Test Cases 

Check whether the output mathches the following cases :

| Indicator | _v<sub>j,t</sub>_ | _f<sub>j,t</sub>_ | | _N<sub>j</sub>_ | _F<sub>j</sub>_ | | _I<sub>j,t</sub>_ |
| --- | ---: | ---: | --- | ---: | ---: | --- | ---: |
| C1 | 2 | 1 | | 3 | yes=1 | | 66.67 |
| C2 | `no data` | `no data` | | 3 | yes=1 | | 0.00 |
| C3 | 2 | 0 | | 2 | yes=1 | | 75.00 |
| C4 | 2 | 0 | | 4 | yes=1 | | 37.50 |
| C5 | 0 | `null` | | 2 | yes=1 | | 0.00 |
| C6 | 1 | 0 | | 3 | yes=1 | | 16.67 |
| C7 | 1 | 1 | | 2 | yes=1 | | 50.00 |
| C8 | 3 | N/A | | 4 | no=0 | | 75.00 |

In [9]:
## Test c1 (should be 66.67) - pass
base_df.query('c1_school_closing==2 and c1_flag==1').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
138,Aruba,ABW,NaN,NaN,20200518,2020,May,18,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,66.6667,100.0,100.0,55.5556
139,Aruba,ABW,NaN,NaN,20200519,2020,May,19,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,66.6667,100.0,100.0,55.5556
140,Aruba,ABW,NaN,NaN,20200520,2020,May,20,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,66.6667,100.0,100.0,55.5556
141,Aruba,ABW,NaN,NaN,20200521,2020,May,21,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,66.6667,100.0,100.0,55.5556
142,Aruba,ABW,NaN,NaN,20200522,2020,May,22,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,66.6667,100.0,100.0,55.5556


In [10]:
## Test c2 (should be 0.00) - pass
base_df.query('c2_workplace_closing != c2_workplace_closing and c2_flag != c2_flag', engine='python').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
762,Aruba,ABW,NaN,NaN,20220201,2022,February,1,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
763,Aruba,ABW,NaN,NaN,20220202,2022,February,2,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
764,Aruba,ABW,NaN,NaN,20220203,2022,February,3,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
765,Aruba,ABW,NaN,NaN,20220204,2022,February,4,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
766,Aruba,ABW,NaN,NaN,20220205,2022,February,5,NaN,NaN,...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
## Test c3 (should be 75.00) - pass
base_df.query('c3_cancel_public_events==2 and c3_flag==0').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
254,Aruba,ABW,NaN,NaN,20200911,2020,September,11,2.0,1.0,...,3.0,66.6667,66.6667,75.0,87.5,0.0,66.6667,0.0,75.0,48.6111
255,Aruba,ABW,NaN,NaN,20200912,2020,September,12,2.0,1.0,...,3.0,66.6667,66.6667,75.0,87.5,0.0,66.6667,0.0,75.0,48.6111
256,Aruba,ABW,NaN,NaN,20200913,2020,September,13,2.0,1.0,...,3.0,66.6667,66.6667,75.0,87.5,0.0,66.6667,0.0,75.0,48.6111
257,Aruba,ABW,NaN,NaN,20200914,2020,September,14,2.0,1.0,...,3.0,66.6667,66.6667,75.0,87.5,0.0,66.6667,0.0,75.0,48.6111
258,Aruba,ABW,NaN,NaN,20200915,2020,September,15,2.0,1.0,...,3.0,66.6667,66.6667,75.0,87.5,0.0,66.6667,0.0,75.0,48.6111


In [12]:
## Test c4 (should be 37.50) - pass
base_df.query('c4_restrictions_on_gatherings==2 and c4_flag==0').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
1886,Angola,AGO,NaN,NaN,20201118,2020,November,18,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407
1887,Angola,AGO,NaN,NaN,20201119,2020,November,19,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407
1888,Angola,AGO,NaN,NaN,20201120,2020,November,20,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407
1889,Angola,AGO,NaN,NaN,20201121,2020,November,21,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407
1890,Angola,AGO,NaN,NaN,20201122,2020,November,22,2.0,1.0,...,4.0,66.6667,66.6667,50.0,37.5,50.0,33.3333,75.0,100.0,53.2407


In [13]:
## Test c5 (should be 0.00) - pass
base_df.query('c5_close_public_transport==0 and c5_flag.isnull()').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
0,Aruba,ABW,NaN,NaN,20200101,2020,January,1,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,ABW,NaN,NaN,20200102,2020,January,2,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,ABW,NaN,NaN,20200103,2020,January,3,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,ABW,NaN,NaN,20200104,2020,January,4,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,ABW,NaN,NaN,20200105,2020,January,5,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
## Test c6 (should be 16.67) - pass
base_df.query('c6_stay_at_home_requirements==1 and c6_flag==0').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
3982,United Arab Emirates,ARE,NaN,NaN,20200313,2020,March,13,3.0,1.0,...,3.0,100.0,16.6667,0.0,0.0,0.0,16.6667,0.0,75.0,23.1482
3983,United Arab Emirates,ARE,NaN,NaN,20200314,2020,March,14,3.0,1.0,...,3.0,100.0,16.6667,0.0,0.0,0.0,16.6667,0.0,75.0,23.1482
3984,United Arab Emirates,ARE,NaN,NaN,20200315,2020,March,15,3.0,1.0,...,3.0,100.0,16.6667,0.0,0.0,0.0,16.6667,0.0,75.0,23.1482
3985,United Arab Emirates,ARE,NaN,NaN,20200316,2020,March,16,3.0,1.0,...,3.0,100.0,16.6667,100.0,0.0,0.0,16.6667,0.0,75.0,34.2593
3986,United Arab Emirates,ARE,NaN,NaN,20200317,2020,March,17,3.0,1.0,...,3.0,100.0,16.6667,100.0,0.0,0.0,16.6667,0.0,75.0,34.2593


In [15]:
## Test c7 (should be 50.00) - pass
base_df.query('c7_restrictions_on_internal_movement==1 and c7_flag==1').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
151,Aruba,ABW,NaN,NaN,20200531,2020,May,31,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,33.3333,50.0,100.0,46.2963
152,Aruba,ABW,NaN,NaN,20200601,2020,June,1,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,33.3333,50.0,100.0,46.2963
153,Aruba,ABW,NaN,NaN,20200602,2020,June,2,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,33.3333,50.0,100.0,46.2963
154,Aruba,ABW,NaN,NaN,20200603,2020,June,3,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,33.3333,50.0,100.0,46.2963
155,Aruba,ABW,NaN,NaN,20200604,2020,June,4,2.0,1.0,...,4.0,66.6667,66.6667,100.0,0.0,0.0,33.3333,50.0,100.0,46.2963


In [16]:
## Test c8 (should be 75.00) - pass
base_df.query('c8_international_travel_controls==3').head(5)

,countryname,countrycode,regionname,regioncode,date,year,month,day,c1_school_closing,c1_flag,...,c8_international_travel_controls,c1_index,c2_index,c3_index,c4_index,c5_index,c6_index,c7_index,c8_index,closure_index
161,Aruba,ABW,NaN,NaN,20200610,2020,June,10,1.0,1.0,...,3.0,33.3333,33.3333,100.0,0.0,0.0,33.3333,50.0,75.0,36.1111
162,Aruba,ABW,NaN,NaN,20200611,2020,June,11,1.0,1.0,...,3.0,33.3333,33.3333,100.0,0.0,0.0,33.3333,50.0,75.0,36.1111
163,Aruba,ABW,NaN,NaN,20200612,2020,June,12,1.0,1.0,...,3.0,33.3333,33.3333,100.0,0.0,0.0,33.3333,50.0,75.0,36.1111
164,Aruba,ABW,NaN,NaN,20200613,2020,June,13,1.0,1.0,...,3.0,33.3333,33.3333,100.0,0.0,0.0,33.3333,50.0,75.0,36.1111
165,Aruba,ABW,NaN,NaN,20200614,2020,June,14,1.0,1.0,...,3.0,33.3333,33.3333,100.0,0.0,0.0,33.3333,50.0,75.0,36.1111
